# Preprocess data

#### Insert image train/validation data to `data/train` and `data/val` folders respectively 
#### Convert every image to `.jpeg` format (for tfrecord decode error prevention)

In [1]:
!python preprocess/to_jpeg.py --folderPath data/train
!python preprocess/to_jpeg.py --folderPath data/val

#### Generate `.csv` file of annotations for each object on each image

In [2]:
!python preprocess/xml_to_csv.py -i data/train -o data/train_labels.csv
!python preprocess/xml_to_csv.py -i data/val -o data/val_labels.csv

#### Generate `.tfrecord` files for tran/validation data to be fed into a model

In [3]:
!python preprocess/facemask_tfrecord.py --output_file data/facemask_val.tfrecord --classes data/facemask.names --data_path data/val --csv_path data/val_labels.csv
!python preprocess/facemask_tfrecord.py --output_file data/facemask_train.tfrecord --classes data/facemask.names --data_path data/train --csv_path data/train_labels.csv

# Modelling/Training

#### Download tiny-YOLO pretrained weights and transform to `.tf` format

In [4]:
!wget https://pjreddie.com/media/files/yolov3-tiny.weights -O data/yolov3-tiny.weights
!python convert.py --weights ./data/yolov3-tiny.weights --output ./checkpoints/face_mask_detection.tf --tiny

### Training parameters
--dataset `.tfrecord` dataset path for train set

--val_dataset `.tfrecord` dataset path for validation set

--weights `Tiny YOLO model weights path`

--classes File path containing class names 

--transfer transfer type (for transfer learning with different output shape we need 'darknet')

--epochs Number of epochs

--num_classes Number of classes in our model

--weights_num_classes Number of classes of model that we're transfering weights from

--tiny True/False whether we use tiny version of YOLO or not

In [5]:
!python train.py --dataset data/facemask_train.tfrecord --val_dataset data/facemask_val.tfrecord \
    --weights checkpoints/face_mask_detection.tf --classes data/facemask.names --mode fit \
    --transfer darknet --epochs 20 --num_classes 2 --weights_num_classes 80 --tiny True